In [13]:
#! pip install gradio
#! pip install shap
#! pip install xgboost
#! pip install datasetshttp://localhost:8888/notebooks/Desktop/Machinefy%20Internship/Untitled-3.ipynb#

In [2]:
import random
import numpy as np
import gradio as gr
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import shap
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score
#from datasets import load_dataset
from sklearn.model_selection import train_test_split

In [3]:
#matplotlib.use("Agg")
dataset = pd.read_csv(r"//Users//omniaelmenshawy//Desktop///a7lafinal.csv")

In [4]:
dataset.drop(['Loan5'],inplace = True, axis = 1)
dataset.drop(['Loan6'],inplace = True, axis = 1)
dataset.drop(['Loan7'],inplace = True, axis = 1)
dataset.drop(['Loan8'],inplace = True, axis = 1)
dataset.drop(['Loan9'],inplace = True, axis = 1)
dataset.drop(['Credit_Mix'],inplace = True, axis = 1)

In [5]:
#dataset = pd.read_csv("/Users/omniaelmenshawy/Desktop//ok.csv")

In [6]:
dataset.columns

Index(['Age', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts',
       'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan',
       'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
       'Num_Credit_Inquiries', 'Outstanding_Debt', 'Credit_Utilization_Ratio',
       'Credit_History_Age', 'Total_EMI_per_month', 'Amount_invested_monthly',
       'Monthly_Balance', 'Loan1', 'Loan2', 'Loan3', 'Loan4', 'Customer_ID',
       'Month', 'Occupation', 'Payment_of_Min_Amount', 'Payment_Behaviour',
       'Credit_Score'],
      dtype='object')

In [7]:
df_testing=dataset.copy(deep=True)

from sklearn.utils import shuffle

df2=shuffle(df_testing[df_testing['Credit_Score']== "Standard"][:25000])

df1=shuffle(df_testing[df_testing['Credit_Score']=="Poor"][:20000])

df0=shuffle(df_testing[df_testing['Credit_Score']=="Good"])

dataset=pd.concat([df2, df1,df0])

dataset=dataset.reset_index(drop=True)

In [8]:
dataset.replace('Standard'  ,2,inplace=True)
dataset.replace('Poor',1,inplace=True)
dataset.replace('Good'   ,0,inplace=True)

In [9]:
dataset.head()

,Age,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,...,Loan1,Loan2,Loan3,Loan4,Customer_ID,Month,Occupation,Payment_of_Min_Amount,Payment_Behaviour,Credit_Score
0,20.0,1892178.0,1403.815000,6.0,9.0,29.0,6.0,27.0,16.0,27.0,...,9.0,6.0,9.0,3.0,CUS_0x53e9,February,Journalist,Yes,Low_spent_Small_value_payments,2
1,27.0,8546163.0,7210.802500,5.0,7.0,11.0,0.0,20.0,14.0,4.0,...,3.0,0.0,0.0,0.0,CUS_0x44c8,April,Architect,Yes,Low_spent_Medium_value_payments,2
2,85.0,1516951.0,1458.404000,3.0,7.0,12.0,4.0,25.0,22.0,12.0,...,2.0,3.0,6.0,6.0,CUS_0x48dc,August,Developer,Yes,Low_spent_Large_value_payments,2
3,29.0,7167759.0,5889.132500,5.0,6.0,16.0,0.0,1.0,11.0,14.0,...,3.0,0.0,0.0,0.0,CUS_0x9641,April,Manager,No,High_spent_Medium_value_payments,2
4,31.0,833116.0,7081.633333,9.0,7.0,27.0,8.0,41.0,19.0,21.0,...,8.0,2.0,6.0,5.0,CUS_0x856c,January,Journalist,Yes,High_spent_Large_value_payments,2


In [10]:
y_train = dataset.pop("Credit_Score")
X_train = dataset.copy(deep=True)
X_train.drop("Customer_ID",inplace = True, axis = 1)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_train,y_train, test_size = 0.2, random_state=44)

In [12]:
X_train.shape

(50262, 25)

In [13]:
X_train.head()

,Age,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,...,Amount_invested_monthly,Monthly_Balance,Loan1,Loan2,Loan3,Loan4,Month,Occupation,Payment_of_Min_Amount,Payment_Behaviour
56749,41.0,3521852.0,3195.876667,1.0,2.0,3.0,2.0,10.0,12.0,4.0,...,46.362454,474.542022,2.0,9.0,0.0,0.0,March,Manager,No,High_spent_Large_value_payments
7313,36.0,1994435.0,1819.029167,8.0,9.0,30.0,5.0,25.0,21.0,4.0,...,105.379312,302.186731,8.0,5.0,8.0,6.0,April,Doctor,Yes,Low_spent_Large_value_payments
5000,16.0,10352745.0,861.728750,3.0,5.0,6.0,7.0,27.0,17.0,15.0,...,102.841486,236.702012,6.0,5.0,8.0,6.0,March,Doctor,Yes,Low_spent_Small_value_payments
33364,17.0,7434644.0,6134.536667,13.0,10.0,25.0,8.0,62.0,18.0,6.0,...,357.166359,232.814037,3.0,9.0,6.0,1.0,March,Engineer,Yes,Low_spent_Medium_value_payments
23657,32.0,2154693.0,1561.577500,5.0,6.0,20.0,1.0,31.0,19.0,30.0,...,171.038564,265.241068,6.0,0.0,0.0,0.0,March,Accountant,No,Low_spent_Small_value_payments


In [14]:
def categorical_to_num(df):
    #set_of_ids=set(df['Customer_ID'])
    #dict_of_ids=dict(zip(set_of_ids,range(1,len(set_of_ids)+1)))
    #df['Customer_ID']=df['Customer_ID'].map(dict_of_ids)
    
    set_of_Month=set(df['Month'])
    dict_of_Month=dict(zip(set_of_Month,range(1,len(set_of_Month)+1)))
    df['Month']=df['Month'].map(dict_of_Month)
    
    set_of_Occupation=set(df['Occupation'])
    dict_of_Occupation=dict(zip(set_of_Occupation,range(1,len(set_of_Occupation)+1)))
    df['Occupation']=df['Occupation'].map(dict_of_Occupation)
    
    set_of_pay=set(df['Payment_of_Min_Amount'])
    set_of_pay=dict(zip(set_of_pay,range(1,len(set_of_pay)+1)))
    df['Payment_of_Min_Amount']=df['Payment_of_Min_Amount'].map(set_of_pay)
    
    set_of_behv=set(df['Payment_Behaviour'])
    set_of_behv=dict(zip(set_of_behv,range(1,len(set_of_behv)+1)))
    df['Payment_Behaviour']=df['Payment_Behaviour'].map(set_of_behv)
    return df

In [15]:
X_train=categorical_to_num(X_train)
X_test=categorical_to_num(X_test)

In [16]:
knn_classifier = KNeighborsClassifier(n_neighbors=3)
knn_classifier.fit(X_train, y_train)
print('the score on train dataset is') 
print(knn_classifier.score(X_train, y_train))
y_pred_knn = knn_classifier.predict(X_test)
print("Test Accuracy : ", accuracy_score(y_test, y_pred_knn))

the score on train dataset is
0.903644900720226
Test Accuracy :  0.8183192742320547


In [17]:
X_train.head()


,Age,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,...,Amount_invested_monthly,Monthly_Balance,Loan1,Loan2,Loan3,Loan4,Month,Occupation,Payment_of_Min_Amount,Payment_Behaviour
56749,41.0,3521852.0,3195.876667,1.0,2.0,3.0,2.0,10.0,12.0,4.0,...,46.362454,474.542022,2.0,9.0,0.0,0.0,7,9,1,5
7313,36.0,1994435.0,1819.029167,8.0,9.0,30.0,5.0,25.0,21.0,4.0,...,105.379312,302.186731,8.0,5.0,8.0,6.0,2,13,2,2
5000,16.0,10352745.0,861.728750,3.0,5.0,6.0,7.0,27.0,17.0,15.0,...,102.841486,236.702012,6.0,5.0,8.0,6.0,7,13,2,7
33364,17.0,7434644.0,6134.536667,13.0,10.0,25.0,8.0,62.0,18.0,6.0,...,357.166359,232.814037,3.0,9.0,6.0,1.0,7,16,2,3
23657,32.0,2154693.0,1561.577500,5.0,6.0,20.0,1.0,31.0,19.0,30.0,...,171.038564,265.241068,6.0,0.0,0.0,0.0,7,14,1,7


In [18]:
def predict(*args):
    ans={2:'Standard',1:'Poor',0:'Good'}
    df = pd.DataFrame([args], columns=X_train.columns)
    #df = df.astype({col: "category" for col in categorical_columns})
    #pos_pred = model_xgb.predict(xgb.DMatrix(df, enable_categorical=True))
    df=categorical_to_num(df)
    y_predict=knn_classifier.predict(df)
    pos_pred=int(y_predict)
    return ans[pos_pred]

In [19]:
dataset.columns.shape

(26,)

In [20]:
unique_ID = sorted(dataset["Customer_ID"].unique())
unique_Month = sorted(dataset["Month"].unique())
unique_Occupation = sorted(dataset["Occupation"].unique())
#unique_CreditMix = sorted(X_train["Credit_Mix"].unique())
unique_Payment_of_min = sorted(dataset["Payment_of_Min_Amount"].unique())
unique_Behaviour = sorted(dataset["Payment_Behaviour"].unique())

In [24]:
import numpy as np
import gradio as gr

def Numerical(x):
    return x[::-1]

def Categorical(x):
    return np.fliplr(x)

with gr.Blocks() as demo:
    gr.Markdown("Credit Score Classification")
    gr.Markdown("Managment Department")
    with gr.Tabs():
        with gr.TabItem("Numerical Values"):
            with gr.Row():
                with gr.Column():
                    Age = gr.Slider(label="Age", minimum=17, maximum=90, step=1, randomize=True)
                    Annual_Income = gr.Slider(label="Annual Income", minimum=20000, maximum=20000000, step=1, randomize=True)

                    Monthly_Inhand_Salary = gr.Slider(label="Monthly Inhand Salary", minimum=200, maximum=20000, step=1, randomize=True)

                    Num_Bank_Accounts = gr.Slider(label="Numumber of  Bank Accounts", minimum=0, maximum=13, step=1, randomize=True)

                    Num_Credit_Card = gr.Slider(label="Num Credit Card", minimum=0, maximum=15, step=1, randomize=True)

                    Interest_Rate = gr.Slider(label="Interest Rate", minimum=1, maximum=40, step=1, randomize=True)

                    Num_of_Loan = gr.Slider(label="Num of Loan", minimum=0, maximum=15, step=1, randomize=True)

                    Delay_from_due_date = gr.Slider(label="Delay from due date", minimum=0, maximum=100, step=1, randomize=True)
                    Num_of_Delayed_Payment = gr.Slider(label="Num of Delayed Payment", minimum=0, maximum=100, step=1, randomize=True)

                    Changed_Credit_Limit = gr.Slider(label="Changed Credit Limit", minimum=0, maximum=100, step=1, randomize=True)

                    Num_Credit_Inquiries = gr.Slider(label="Num Credit Inquiries", minimum=0, maximum=30, step=1, randomize=True)

                    Outstanding_Debt = gr.Slider(label="Outstanding Debt", minimum=0, maximum=5000, step=1, randomize=True)

                    Credit_Utilization_Ratio = gr.Slider(label="Credit Utilization_Ratio", minimum=20, maximum=60, step=1, randomize=True)

                    Credit_History_Age = gr.Slider(label="Credit History Age", minimum=0, maximum=35, step=1, randomize=True)

                    Total_EMI_per_month = gr.Slider(label="Total EMI per month", minimum=0, maximum=370, step=1, randomize=True)

                    Amount_invested_monthly = gr.Slider(label="Amount invested monthly", minimum=0, maximum=1000, step=1, randomize=True)

                    Monthly_Balance = gr.Slider(label="Monthly Balance", minimum=0, maximum=2000, step=1, randomize=True)
                    Loan1 = gr.Slider(label="Loan1", minimum=0, maximum=9, step=1, randomize=True)
                    Loan2 = gr.Slider(label="Loan2", minimum=0, maximum=9, step=1, randomize=True)


                    Loan3 = gr.Slider(label="Loan3", minimum=0, maximum=9, step=1, randomize=True)

                    Loan4 = gr.Slider(label="Loan4", minimum=0, maximum=9, step=1, randomize=True)




                    
        with gr.TabItem("categorical Values"):
            with gr.Row():
                with gr.Column():
                    Customer_ID = gr.Dropdown(
                    label="Customer ID",
                    choices=unique_ID,
                    value=lambda: random.choice(unique_ID),
            )
                    Month = gr.Dropdown(
                    label="Month",
                    choices=unique_Month,
                    value=lambda: random.choice(unique_Month),
            )

                    Occupation = gr.Dropdown(
                    label="Occupation",
                    choices=unique_Occupation,               
                    value=lambda: random.choice(unique_Occupation),
            )


                    Payment_of_Min_Amount = gr.Dropdown(
                    label="Payment of Minimum Amount",
                    choices=unique_Payment_of_min,
                    value=lambda: random.choice(unique_Payment_of_min)
            )
            
                    Payment_Behaviour = gr.Dropdown(
                    label="Payment Behaviour",
                    choices=unique_Behaviour,
                    value=lambda: random.choice(unique_Behaviour)
                
            )
            


        with gr.Column():
            label = gr.Label()
            
            with gr.Row():
                predict_btn = gr.Button(value="Predict")
    
            predict_btn.click(
                predict,
                inputs=[
                    Age,
                    Annual_Income,
                    Monthly_Inhand_Salary,
                    Num_Bank_Accounts,
                    Num_Credit_Card,
                    Interest_Rate,
                    Num_of_Loan,
                    Delay_from_due_date,
                    Num_of_Delayed_Payment,
                    Changed_Credit_Limit,
                    Num_Credit_Inquiries,
                    Outstanding_Debt,
                    Credit_Utilization_Ratio,
                    Credit_History_Age,
                    Total_EMI_per_month,
                    Amount_invested_monthly, 
                    Monthly_Balance,     
                    Loan1,                                                                                                                                                                                                                                                                                     
                    Loan2,
                    Loan3,
                    Loan4,
                    #Customer_ID,
                    Month,
                    Occupation,
                    #Credit_Mix,
                    Payment_of_Min_Amount,                                              
                    Payment_Behaviour,                                                                                                                                                                                                                                    
                    
                ],
                outputs=[label])


demo.launch()




Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB
Running on local URL:  http://127.0.0.1:7865/

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x7fdf7c26e340>, 'http://127.0.0.1:7865/', None)

In [24]:
x_test.shape

(12566, 26)

In [32]:
dataset.columns

Index(['Age', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts',
       'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan',
       'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
       'Num_Credit_Inquiries', 'Outstanding_Debt', 'Credit_Utilization_Ratio',
       'Credit_History_Age', 'Total_EMI_per_month', 'Amount_invested_monthly',
       'Monthly_Balance', 'Loan1', 'Loan2', 'Loan3', 'Loan4', 'Customer_ID',
       'Month', 'Occupation', 'Payment_of_Min_Amount', 'Payment_Behaviour'],
      dtype='object')